In [1]:
# =========================================================
# PROCESS VIDEOS WITH YOLOv5 + SIMPLE TRACKER
# =========================================================

import os
import cv2
import csv
import time
import math
import torch
from pathlib import Path

# ======================
# CONFIG
# ======================
VIDEOS_DIR = "/Users/diptamulyapratama/Downloads/BDDA/test/camera_videos"
OUTPUT_DIR = "/Users/diptamulyapratama/Downloads/OUTPUT_SML_FIX_1"
LOG_DIR = os.path.join(OUTPUT_DIR, "logs")

In [2]:
SAMPLE_FPS = 10
IOU_TRACKER_THRESHOLD = 0.3
MIN_BOX_AREA = 8000
MIN_SPEED_PX = 3
SPEED_SMOOTH_FRAMES = 5

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)


In [3]:

# ======================
# LOAD YOLOv5
# ======================
model = torch.hub.load(
    'ultralytics/yolov5',
    'yolov5n',
    pretrained=True
)
model.conf = 0.4
model.iou = 0.45
model.eval()

Using cache found in /Users/diptamulyapratama/.cache/torch/hub/ultralytics_yolov5_master


requirements: Ultralytics requirements ['pillow>=10.3.0', 'tqdm>=4.66.3'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/5.3 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/5.3 MB ? eta -:--:--
   ━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.3/5.3 MB ? eta -:--:--
   ━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.5/5.3 MB 2.1 MB/s eta 0:00:03
   ━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.5/5.3 MB 2.1 MB/s eta 0:00:03
   ━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.8/5.3 MB 1.1 MB/s eta 0:00:05
   ━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/5.3 MB 906.0 kB/s eta 0:00:05
   ━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/5.3 MB 1.3 MB/s eta 0:00:03
   ━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/5.3 MB 1.3 MB/s eta 0:00:03
   ━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/5.3 MB 1.1 MB/s eta 0:00:04
   ━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/5.3 MB 1.1 MB/s eta 0:00:04
   ━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/5.3 MB 1.1 MB/s eta 0:00:

YOLOv5 🚀 2025-12-19 Python-3.11.7 torch-2.2.2 CPU

Fusing layers... 
[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients, 4.5 GFLOPs
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 16, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
  

In [4]:
# ======================
# TRACKER
# ======================
class SimpleTracker:
    def __init__(self, iou_threshold=0.3):
        self.tracks = {}
        self.next_id = 1
        self.iou_th = iou_threshold
        self.history = {}
        self.event_counters = {}

    def update(self, detections):
        assigned = set()
        new_tracks = {}

        for d in detections:
            best_id, best_iou = None, 0
            for tid, bbox in self.tracks.items():
                iou = compute_iou(d, bbox)
                if iou > best_iou and iou >= self.iou_th:
                    best_iou = iou
                    best_id = tid

            if best_id is not None and best_id not in assigned:
                new_tracks[best_id] = d
                assigned.add(best_id)
            else:
                new_tracks[self.next_id] = d
                self.next_id += 1

        for tid, bbox in new_tracks.items():
            c = bbox_center(bbox)
            self.history.setdefault(tid, []).append((c, time.time()))
            self.history[tid] = self.history[tid][-30:]
            self.event_counters.setdefault(tid, {'tail': 0})

        self.tracks = new_tracks
        return self.tracks

In [5]:
# ======================
# UTILS
# ======================
def compute_iou(a, b):
    ax1, ay1, ax2, ay2 = a
    bx1, by1, bx2, by2 = b
    ix1, iy1 = max(ax1, bx1), max(ay1, by1)
    ix2, iy2 = min(ax2, bx2), min(ay2, by2)
    iw, ih = max(0, ix2 - ix1), max(0, iy2 - iy1)
    inter = iw * ih
    union = (ax2-ax1)*(ay2-ay1) + (bx2-bx1)*(by2-by1) - inter + 1e-6
    return inter / union

def bbox_center(b):
    x1,y1,x2,y2 = b
    return ((x1+x2)/2, (y1+y2)/2)

def pixel_distance(a,b):
    return math.hypot(a[0]-b[0], a[1]-b[1])

def estimate_speed(hist):
    if len(hist) < 2:
        return 0
    (c1,t1),(c2,t2) = hist[-2], hist[-1]
    return pixel_distance(c1,c2)/max(t2-t1,1e-6)

In [6]:
# ======================
# YOLOv5 DETECTION
# ======================
def yolo5_detect(frame):
    results = model(frame)
    detections = []

    for *xyxy, conf, cls in results.xyxy[0]:
        x1,y1,x2,y2 = map(float, xyxy)
        area = (x2-x1)*(y2-y1)
        if area < MIN_BOX_AREA:
            continue
        detections.append([x1,y1,x2,y2])

    return detections

# ======================
# BEHAVIOR DETECTORS
# ======================
def detect_tailgating(tracks, tracker, frame_h,
                      min_speed=20, max_gap_ratio=0.08, stable_frames=5):

    events = []
    items = list(tracks.items())

    for i,(id1,b1) in enumerate(items):
        spd1 = estimate_speed(tracker.history.get(id1, [])[-SPEED_SMOOTH_FRAMES:])
        if spd1 < min_speed:
            continue

        c1 = bbox_center(b1)
        for j,(id2,b2) in enumerate(items):
            if i == j: continue
            c2 = bbox_center(b2)

            if c2[1] < c1[1]:
                gap = c1[1] - c2[1]
                if gap < frame_h * max_gap_ratio:
                    tracker.event_counters[id1]['tail'] += 1
                    if tracker.event_counters[id1]['tail'] >= stable_frames:
                        events.append((id1,id2,gap))
                else:
                    tracker.event_counters[id1]['tail'] = 0

    return events

def detect_speed_events(tracker, threshold=80):
    events = []
    for tid,hist in tracker.history.items():
        spd = estimate_speed(hist[-SPEED_SMOOTH_FRAMES:])
        if spd > threshold:
            events.append((tid, spd))
    return events

In [7]:
# ======================
# MAIN VIDEO PROCESS
# ======================
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS) or 30
    step = max(1, int(round(fps / SAMPLE_FPS)))

    w = int(cap.get(3))
    h = int(cap.get(4))

    name = Path(video_path).stem
    out_vid = cv2.VideoWriter(
        os.path.join(OUTPUT_DIR, f"{name}_annotated.mp4"),
        cv2.VideoWriter_fourcc(*'mp4v'),
        SAMPLE_FPS,
        (w,h)
    )

    csvf = open(os.path.join(LOG_DIR,f"{name}_events.csv"),'w',newline='')
    writer = csv.writer(csvf)
    writer.writerow(['video','frame','time','event','id1','id2','metric'])

    tracker = SimpleTracker(IOU_TRACKER_THRESHOLD)
    idx = sidx = 0

    while True:
        ret, frame = cap.read()
        if not ret: break
        if idx % step != 0:
            idx += 1
            continue

        dets = yolo5_detect(frame)
        tracks = tracker.update(dets)

        tails = detect_tailgating(tracks, tracker, h)
        speeds = detect_speed_events(tracker)

        t = idx / fps
        for a,b,g in tails:
            writer.writerow([name,sidx,f"{t:.2f}","tailgating",a,b,int(g)])
        for tid,spd in speeds:
            writer.writerow([name,sidx,f"{t:.2f}","speed",tid,"",f"{spd:.1f}"])

        vis = frame.copy()
        for tid,(x1,y1,x2,y2) in tracks.items():
            cv2.rectangle(vis,(int(x1),int(y1)),(int(x2),int(y2)),(0,255,0),2)
            cv2.putText(vis,f"ID:{tid}",(int(x1),int(y1)-5),
                        cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,255),1)

        out_vid.write(vis)
        idx += 1
        sidx += 1

    cap.release()
    out_vid.release()
    csvf.close()
    print(f"[DONE] {video_path}")

In [8]:
# ======================
# RUN ALL VIDEOS
# ======================
def main():
    vids = [os.path.join(VIDEOS_DIR,f) for f in os.listdir(VIDEOS_DIR)
            if f.lower().endswith(('.mp4','.avi','.mov','.mkv'))]
    for v in vids:
        process_video(v)

if __name__ == "__main__":
    main()

[DONE] /Users/diptamulyapratama/Downloads/BDDA/test/camera_videos/544.mp4
[DONE] /Users/diptamulyapratama/Downloads/BDDA/test/camera_videos/1013.mp4
[DONE] /Users/diptamulyapratama/Downloads/BDDA/test/camera_videos/593.mp4
[DONE] /Users/diptamulyapratama/Downloads/BDDA/test/camera_videos/1588.mp4
[DONE] /Users/diptamulyapratama/Downloads/BDDA/test/camera_videos/791.mp4
[DONE] /Users/diptamulyapratama/Downloads/BDDA/test/camera_videos/1403.mp4
[DONE] /Users/diptamulyapratama/Downloads/BDDA/test/camera_videos/829.mp4
[DONE] /Users/diptamulyapratama/Downloads/BDDA/test/camera_videos/1417.mp4
[DONE] /Users/diptamulyapratama/Downloads/BDDA/test/camera_videos/424.mp4
[DONE] /Users/diptamulyapratama/Downloads/BDDA/test/camera_videos/1167.mp4
[DONE] /Users/diptamulyapratama/Downloads/BDDA/test/camera_videos/1601.mp4
[DONE] /Users/diptamulyapratama/Downloads/BDDA/test/camera_videos/1614.mp4
[DONE] /Users/diptamulyapratama/Downloads/BDDA/test/camera_videos/343.mp4
[DONE] /Users/diptamulyapratama

KeyboardInterrupt: 